In [1]:
library(Signac)
library(Seurat)
library(ggplot2)
library(dplyr)
library(patchwork)
library(hdf5r)
library(readr)
library(GenomeInfoDb)
library(EnsDb.Hsapiens.v86)

Loading required package: SeuratObject

Loading required package: sp

‘SeuratObject’ was built with package ‘Matrix’ 1.6.4 but the current
version is 1.6.5; it is recomended that you reinstall ‘SeuratObject’ as
the ABI for ‘Matrix’ may have changed


Attaching package: ‘SeuratObject’


The following object is masked from ‘package:base’:

    intersect



Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


Loading required package: BiocGenerics


Attaching package: ‘BiocGenerics’


The following objects are masked from ‘package:dplyr’:

    combine, intersect, setdiff, union


The following object is masked from ‘package:SeuratObject’:

    intersect


The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs


The following objects are masked from ‘package:base’:

    anyDuplicated, aperm, append, as.data.frame, b

In [2]:
library(future)
plan("multicore", workers = 32)
options(future.globals.maxSize = 8000 * 1024^2)


Attaching package: 'future'


The following object is masked from 'package:AnnotationFilter':

    value




- 读取scATAC-seq数据，创建 seuart 对象
- 注释基因
- QC（绘图、保存）
- 归一化、降维
- 聚类、UMAP
- 基因活性矩阵
- 保存（./data/batch.rds）

In [3]:
batches <- c("10N", '10T', '12N', '12T', '13T', '19T', '20T', '21T', '22T')

In [4]:
create_seurat_object <- function(peak_bc_filePath, fragments_filePath) {
    print("Read scATAC-seq data")
    mtx_path <- paste(peak_bc_filePath, "matrix.mtx", sep = '/')
    feature_path <- paste(peak_bc_filePath, "peaks.bed", sep = '/')
    barcode_path <- paste(peak_bc_filePath, "barcodes.tsv", sep = '/')

    features <- readr::read_tsv(feature_path, col_names = F) %>% tidyr::unite(feature)
    barcodes <- readr::read_tsv(barcode_path, col_names = F) %>% tidyr::unite(barcode)

    mtx <- Matrix::readMM(mtx_path) %>%
    magrittr::set_rownames(features$feature) %>%
    magrittr::set_colnames(barcodes$barcode)

    chrom_assay <- CreateChromatinAssay(
        counts = mtx,
        sep = c('_', '_'),
        fragments = fragments_filePath,
        min.cells = 10,
        min.features = 200
    )
    atac <- CreateSeuratObject(
        counts = chrom_assay,
        assay = 'peaks')
    print(atac)

    return(atac)
}

In [5]:
genome_annotations <- function(atac) {
    print("Read genome annotations")
    # 加载注释 EnsDb.Hsapiens.v86-hg38 ,EnsDb.Hsapiens.v75-hg19
    annotations <- GetGRangesFromEnsDb(ensdb = EnsDb.Hsapiens.v86)
    seqlevels(annotations) <- paste0('chr', seqlevels(annotations))
    genome(annotations) <- "hg38"
    Annotation(atac) <- annotations
    return(atac)
}

In [6]:
QC_whithout_plot <- function(atac, fragments_filePath) {
    print("QC")
    # QC,不画图
    atac <- NucleosomeSignal(atac)
    atac$nucleosome_group <- ifelse(atac$nucleosome_signal > 4, 'NS > 4', 'NS < 4')
    atac <- TSSEnrichment(object = atac, fast = FALSE)
    atac$high.tss <- ifelse(atac$TSS.enrichment > 3, 'High', 'Low')

    total_fragments <- CountFragments(fragments_filePath)
    rownames(total_fragments) <- total_fragments$CB
    atac$fragments <- total_fragments[colnames(atac), "frequency_count"]

    atac <- FRiP(
        object = atac,
        assay = 'peaks',
        total.fragments = 'fragments'
    )

    atac$blacklist_fraction <- FractionCountsInRegion(
    object = atac, 
    assay = 'peaks',
    regions = blacklist_hg38
    )
    return(atac)
}

In [7]:
Normalize_reduction <- function(atac) {
    print("Normalization")
    atac <- RunTFIDF(atac)
    atac <- FindTopFeatures(atac, min.cutoff = 'q0')
    atac <- RunSVD(atac)
    return(atac)
}

In [8]:
UMAP_clustering <- function(atac) {
    print("UMAP")
    atac <- RunUMAP(object = atac, reduction = 'lsi', dims = 2:30)
    atac <- FindNeighbors(object = atac, reduction = 'lsi', dims = 2:30)
    atac <- FindClusters(object = atac, verbose = FALSE, algorithm = 3, resolution = 0.4)
    return(atac)
}

In [9]:
Create_gene_activity_matrix <- function(atac) {
    print("Create gene activity matrix")
    gene.activities <- GeneActivity(atac)
    # 这里还是命名为RNA，和官方保持一致，试一下，否则好像有bug
    atac[['RNA']] <- CreateAssayObject(counts = gene.activities)
    atac <- NormalizeData(
    object = atac,
    assay = 'RNA',
    normalization.method = 'LogNormalize',
    scale.factor = median(atac$nCount_RNA)
    )
    return(atac)
}

In [10]:
for (batch in batches) {
    print(paste0("start processing batch:",batch))
    batch_number = gsub("\\D", "", batch)
    peak_bc_filePath = paste0("/data/BCY/BCY-pair/", batch_number, "/", batch, "-scATAC","/filtered_peak_bc_matrix")
    fragments_filePath = paste0("/data/BCY/BCY-pair/", batch_number, "/", batch, "-scATAC", "/fragments.tsv.gz")
    filePath = paste0("./data/", batch,  ".rds")
    print(peak_bc_filePath)
    print(fragments_filePath)
    print(filePath)
    
    atac <- create_seurat_object(peak_bc_filePath, fragments_filePath)
    atac <- genome_annotations(atac)
    atac <- QC_whithout_plot(atac, fragments_filePath)
    atac <- Normalize_reduction(atac)
    atac <- UMAP_clustering(atac)
    atac <- Create_gene_activity_matrix(atac)
    

    saveRDS(atac, file = filePath)
}

[1] "start processing batch:10N"
[1] "/data/BCY/BCY-pair/10/10N-scATAC/filtered_peak_bc_matrix"
[1] "/data/BCY/BCY-pair/10/10N-scATAC/fragments.tsv.gz"
[1] "./data/10N.rds"
[1] "Read scATAC-seq data"


Rows: 234255 Columns: 3
── Column specification ────────────────────────────────────────────────────────
Delimiter: "\t"
chr (1): X1
dbl (2): X2, X3

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
Rows: 12160 Columns: 1
── Column specification ────────────────────────────────────────────────────────
Delimiter: "\t"
chr (1): X1

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
Computing hash



An object of class Seurat 
234254 features across 12160 samples within 1 assay 
Active assay: peaks (234254 features, 0 variable features)
 2 layers present: counts, data
[1] "Read genome annotations"


Warning message in .merge_two_Seqinfo_objects(x, y):
"The 2 combined objects have no sequence levels in common. (Use
  suppressWarnings() to suppress this warning.)"
Warning message in .merge_two_Seqinfo_objects(x, y):
"The 2 combined objects have no sequence levels in common. (Use
  suppressWarnings() to suppress this warning.)"
Warning message in .merge_two_Seqinfo_objects(x, y):
"The 2 combined objects have no sequence levels in common. (Use
  suppressWarnings() to suppress this warning.)"
Warning message in .merge_two_Seqinfo_objects(x, y):
"The 2 combined objects have no sequence levels in common. (Use
  suppressWarnings() to suppress this warning.)"
Warning message in .merge_two_Seqinfo_objects(x, y):
"The 2 combined objects have no sequence levels in common. (Use
  suppressWarnings() to suppress this warning.)"
Warning message in .merge_two_Seqinfo_objects(x, y):
"The 2 combined objects have no sequence levels in common. (Use
  suppressWarnings() to suppress this warning.)"
Warn

[1] "QC"


...

Performing TF-IDF normalization

Running SVD

Scaling cell embeddings



[1] "UMAP"


10:52:36 UMAP embedding parameters a = 0.9922 b = 1.112

Found more than one class "dist" in cache; using the first, from namespace 'BiocGenerics'

Also defined by 'spam'

10:52:36 Read 6732 rows and found 29 numeric columns

10:52:36 Using Annoy for neighbor search, n_neighbors = 30

Found more than one class "dist" in cache; using the first, from namespace 'BiocGenerics'

Also defined by 'spam'

10:52:36 Building Annoy index with metric = cosine, n_trees = 50

0%   10   20   30   40   50   60   70   80   90   100%

[----|----|----|----|----|----|----|----|----|----|

*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
|

10:52:37 Writing NN index file to temp file /tmp/RtmphtbYfi/file3f2c927f8931f4

10:52:37 Searching Annoy index using 32 threads, search_k = 3000

10:52:37 Annoy recall = 100%

10:52:57 Commencing smooth kNN distance calibration using 32 threads
 with target n_neighbors = 30

10:53:07 Initializing from normalized Laplaci

[1] "Create gene activity matrix"


Extracting gene coordinates

Extracting reads overlapping genomic regions



In [11]:
# '11T'单独处理
batch = '11T'
print(paste0("start processing batch:",batch))
batch_number = gsub("\\D", "", batch)
peak_bc_filePath = paste0("/data/BCY/BCY-pair/", batch_number, "/", batch_number, "-scATAC","/filtered_peak_bc_matrix")
fragments_filePath = paste0("/data/BCY/BCY-pair/", batch_number, "/", batch_number, "-scATAC", "/fragments.tsv.gz")
filePath = paste0("./data/", batch,  ".rds")
print(peak_bc_filePath)
print(fragments_filePath)
print(filePath)
    
atac <- create_seurat_object(peak_bc_filePath, fragments_filePath)
atac <- genome_annotations(atac)
atac <- QC_whithout_plot(atac, fragments_filePath)
atac <- Normalize_reduction(atac)
atac <- UMAP_clustering(atac)
atac <- Create_gene_activity_matrix(atac)
    

saveRDS(atac, file = filePath)

[1] "start processing batch:11T"
[1] "/data/BCY/BCY-pair/11/11-scATAC/filtered_peak_bc_matrix"
[1] "/data/BCY/BCY-pair/11/11-scATAC/fragments.tsv.gz"
[1] "./data/11T.rds"
[1] "Read scATAC-seq data"


Rows: 155858 Columns: 3
── Column specification ────────────────────────────────────────────────────────
Delimiter: "\t"
chr (1): X1
dbl (2): X2, X3

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
Rows: 8777 Columns: 1
── Column specification ────────────────────────────────────────────────────────
Delimiter: "\t"
chr (1): X1

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
Computing hash



An object of class Seurat 
155766 features across 8777 samples within 1 assay 
Active assay: peaks (155766 features, 0 variable features)
 2 layers present: counts, data
[1] "Read genome annotations"


Warning message in .merge_two_Seqinfo_objects(x, y):
"The 2 combined objects have no sequence levels in common. (Use
  suppressWarnings() to suppress this warning.)"
Warning message in .merge_two_Seqinfo_objects(x, y):
"The 2 combined objects have no sequence levels in common. (Use
  suppressWarnings() to suppress this warning.)"
Warning message in .merge_two_Seqinfo_objects(x, y):
"The 2 combined objects have no sequence levels in common. (Use
  suppressWarnings() to suppress this warning.)"
Warning message in .merge_two_Seqinfo_objects(x, y):
"The 2 combined objects have no sequence levels in common. (Use
  suppressWarnings() to suppress this warning.)"
Warning message in .merge_two_Seqinfo_objects(x, y):
"The 2 combined objects have no sequence levels in common. (Use
  suppressWarnings() to suppress this warning.)"
Warning message in .merge_two_Seqinfo_objects(x, y):
"The 2 combined objects have no sequence levels in common. (Use
  suppressWarnings() to suppress this warning.)"
Warn

[1] "QC"


...

Performing TF-IDF normalization

Running SVD

Scaling cell embeddings



[1] "UMAP"


11:26:18 UMAP embedding parameters a = 0.9922 b = 1.112

Found more than one class "dist" in cache; using the first, from namespace 'BiocGenerics'

Also defined by 'spam'

11:26:18 Read 8777 rows and found 29 numeric columns

11:26:18 Using Annoy for neighbor search, n_neighbors = 30

Found more than one class "dist" in cache; using the first, from namespace 'BiocGenerics'

Also defined by 'spam'

11:26:18 Building Annoy index with metric = cosine, n_trees = 50

0%   10   20   30   40   50   60   70   80   90   100%

[----|----|----|----|----|----|----|----|----|----|

*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
|

11:26:20 Writing NN index file to temp file /tmp/RtmphtbYfi/file3f2c922c7b7d5b

11:26:20 Searching Annoy index using 32 threads, search_k = 3000

11:26:20 Annoy recall = 100%

11:26:24 Commencing smooth kNN distance calibration using 32 threads
 with target n_neighbors = 30

11:26:30 Initializing from normalized Laplaci

[1] "Create gene activity matrix"


Extracting gene coordinates

Extracting reads overlapping genomic regions

